In [2]:
import numpy as np
import transformers
from transformers import Trainer
transformers.logging.set_verbosity_error()
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, DatasetDict, Dataset, load_metric
from sklearn.metrics import confusion_matrix
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding

In [40]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True)

def read_and_tokenize_data(train,test,evaluate):
    d = load_dataset('pandas', data_files={"train":train, "test":test,"evaluate":evaluate})
    for i in d:
        d[i]= d[i].remove_columns(column_names = ['__index_level_0__'])
    d = d.map(tokenize_function, batched=True)
    return d

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

loading file https://huggingface.co/roberta-base/resolve/main/vocab.json from cache at C:\Users\dovyd/.cache\huggingface\transformers\d3ccdbfeb9aaa747ef20432d4976c32ee3fa69663b379deb253ccfce2bb1fdc5.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab
loading file https://huggingface.co/roberta-base/resolve/main/merges.txt from cache at C:\Users\dovyd/.cache\huggingface\transformers\cafdecc90fcab17011e12ac813dd574b4b3fea39da6dd817813efa010262ff3f.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
loading file https://huggingface.co/roberta-base/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/roberta-base/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/roberta-base/resolve/main/tokenizer_config.json from cache at None
loading file https://huggingface.co/roberta-base/resolve/main/tokenizer.json from cache at C:\Users\dovyd/.cache\huggingface\transformers\d53fc0fa09b8342651efd4073d7

#### Reading data

In [41]:
data_dict = read_and_tokenize_data("data/authors_train.pkl",\
                               "data/authors_test.pkl",\
                                   "data/authors_validation.pkl")

data_dict_2 = read_and_tokenize_data("data/imdb_train.pkl",\
                               "data/imdb_test.pkl",\
                                   "data/imdb_validation.pkl")

Using custom data configuration default-e38bfbbbe1600378
Reusing dataset pandas (C:\Users\dovyd\.cache\huggingface\datasets\pandas\default-e38bfbbbe1600378\0.0.0\6197c1e855b639d75a767140856841a562b7a71d129104973fe1962594877ade)


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Using custom data configuration default-2ce70cf2cb0a1aba
Reusing dataset pandas (C:\Users\dovyd\.cache\huggingface\datasets\pandas\default-2ce70cf2cb0a1aba\0.0.0\6197c1e855b639d75a767140856841a562b7a71d129104973fe1962594877ade)


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

#### Training model

In [ ]:
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels = 4)

model_2 = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels = 2)

In [46]:
training_args = TrainingArguments(
    output_dir='results',
    learning_rate=2e-5,
    num_train_epochs=20,
    weight_decay=0.01,
    evaluation_strategy="steps",
    eval_steps = 250,
    save_steps = 250,
    load_best_model_at_end = True
)


metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


trainer = Trainer(
    model=model,
    args = training_args,
    train_dataset=data_dict["train"],
    eval_dataset=data_dict["evaluate"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


trainer_2 = Trainer(
    model=model,
    args = training_args,
    train_dataset=data_dict_2["train"],
    eval_dataset=data_dict_2["evaluate"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer.train()

In [ ]:
trainer_2.train()

The following columns in the training set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 2700
  Num Epochs = 20
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 6760
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 150
  Batch size = 8
Saving model checkpoint to results/checkpoint-250
Configuration saved in results/checkpoint-250/config.json


{'eval_loss': 0.3027671277523041, 'eval_accuracy': 0.92, 'eval_runtime': 1.4398, 'eval_samples_per_second': 104.18, 'eval_steps_per_second': 13.196, 'epoch': 0.74}


Model weights saved in results/checkpoint-250/pytorch_model.bin
tokenizer config file saved in results/checkpoint-250/tokenizer_config.json
Special tokens file saved in results/checkpoint-250/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 150
  Batch size = 8


{'loss': 0.3491, 'learning_rate': 1.85207100591716e-05, 'epoch': 1.48}


Saving model checkpoint to results/checkpoint-500
Configuration saved in results/checkpoint-500/config.json


...

Model weights saved in results/checkpoint-5250/pytorch_model.bin
tokenizer config file saved in results/checkpoint-5250/tokenizer_config.json
Special tokens file saved in results/checkpoint-5250/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 150
  Batch size = 8


{'loss': 0.0, 'learning_rate': 3.7278106508875745e-06, 'epoch': 16.27}


Saving model checkpoint to results/checkpoint-5500
Configuration saved in results/checkpoint-5500/config.json


{'eval_loss': 0.7859207987785339, 'eval_accuracy': 0.92, 'eval_runtime': 1.4347, 'eval_samples_per_second': 104.549, 'eval_steps_per_second': 13.243, 'epoch': 16.27}


Model weights saved in results/checkpoint-5500/pytorch_model.bin
tokenizer config file saved in results/checkpoint-5500/tokenizer_config.json
Special tokens file saved in results/checkpoint-5500/special_tokens_map.json


#### Prediction and metrics

In [44]:
def make_predictions(trainer,test):
    predictions = trainer.predict(test)
    y_true = test["label"]
    y_pred = np.argmax(predictions.predictions, axis=-1)
    confusion_matrix(y_true, y_pred)
    metric = load_metric("accuracy")
    metric.compute(predictions=y_pred, references=y_true)

#### Saving model

In [ ]:
def save_model(trainer,tokenizer,trainer_path,tokenizer_path):
    trainer.save_model(trainer_path)
    tokenizer.save_pretrained(tokenizer_path)
    
save_model(trainer,tokenizer,"models/author_classification/model","models/author_classification/tokenizer")
save_model(trainer_2,tokenizer,"models/sentiment_classification/model","models/sentiment_classification/tokenizer")

#### Loading this model

In [57]:
def load_model(model_path,tokenizer_path):
    model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels = 4)
    tokenizer = RobertaTokenizer.from_pretrained(tokenizer_path)
    trainer = Trainer(model = model)
    return (model, trainer, tokenizer)
    

model, trainer, tokenizer = load_model("models/author_classification/model","models/author_classification/tokenizer")

model_2, trainer_2, _ = load_model("models/sentiment_classification/model","models/sentiment_classification/tokenizer")

loading configuration file models_2/author_classification/model\config.json
Model config RobertaConfig {
  "_name_or_path": "models_2/author_classification/model",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.15.0",
  "type_vocab_s

In [58]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


def read_and_tokenize_data(data):
    d = load_dataset('pandas', data_files=data,split="train")
    d = d.map(tokenize_function, batched=True)
    return d


def make_predictions(trainer,data_path):
    test_data = read_and_tokenize_data(data_path)
    predictions = trainer.predict(test_data)
    y_true = test_data["label"]
    y_pred = np.argmax(predictions.predictions, axis=-1)
    print(confusion_matrix(test_data["label"], y_pred))
    metric = load_metric("accuracy")
    print(metric.compute(predictions=y_pred, references=y_true))



make_predictions(trainer,"data/authors_small_train.pkl")

make_predictions(trainer_2,"data/imdb_small_train.pkl")

Using custom data configuration default-05868a22cc33adaa
Reusing dataset pandas (C:\Users\dovyd\.cache\huggingface\datasets\pandas\default-05868a22cc33adaa\0.0.0\6197c1e855b639d75a767140856841a562b7a71d129104973fe1962594877ade)


  0%|          | 0/1 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: __index_level_0__, text.
***** Running Prediction *****
  Num examples = 90
  Batch size = 8


[[26  0  0  0]
 [ 0  9  0  0]
 [ 0  0  4  0]
 [ 1  1  0 49]]
{'accuracy': 0.9777777777777777}


In [18]:
from transformers import TextClassificationPipeline

In [34]:
pipeline = TextClassificationPipeline(model=model,tokenizer=tokenizer)

pipeline("Classify this!")

[{'label': 'LABEL_2', 'score': 0.7999024391174316}]

In [ ]:
pipeline_2 = TextClassificationPipeline(model=model_2,tokenizer=tokenizer)

pipeline_2("Classify this!")